In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name="llama-3.3-70b-versatile"
)

response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/generative-ai-design-expert/job/R-71324")
page_data = loader.load().pop().page_content

print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Generative AI Design Expert











































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [3]:
from langchain_community.document_loaders import WebBaseLoader
import re

loader = WebBaseLoader("https://careers.nike.com/generative-ai-design-expert/job/R-71324")
raw_data = loader.load().pop().page_content

# Basit temizlik: Gereksiz boşlukları ve satır başlarını temizle
def clean_text(text):
    text = re.sub(r'\n+', '\n', text)  # Çoklu satır boşluklarını temizle
    text = re.sub(r'\s+', ' ', text)   # Fazla boşlukları temizle
    return text.strip()

cleaned_page_data = clean_text(raw_data)

In [4]:
from langchain_core.prompts import PromptTemplate

# Daha profesyonel ve yapılandırılmış şablon
prompt_extract = PromptTemplate.from_template(
    """
    ### ROLE: 
    You are an expert Data Extraction Specialist focused on recruitment and job market analysis. 
    Your task is to parse raw HTML/Text content and extract specific job opportunities.

    ### CONTEXT:
    The following text is scraped from a company's career page. It may contain noise (headers, ads, footers). 
    Identify if there is a specific job posting and extract the details.

    ### INPUT DATA:
    <scraped_content>
    {page_data}
    </scraped_content>

    ### EXTRACTION RULES:
    1. If multiple jobs are found, return them as a list of JSON objects.
    2. If no job posting is found, return an empty list: [].
    3. Ensure the 'description' field is a concise summary of the role (max 3 sentences) to save downstream tokens.
    4. Clean all special characters from the 'skills' list.

    ### OUTPUT FORMAT:
    Return ONLY valid JSON. No preamble, no explanations.
    Schema:
    {{
        "jobs": [
            {{
                "role": "Job title",
                "experience": "Years of experience required or seniority level",
                "skills": ["skill1", "skill2"],
                "description": "Short overview"
            }}
        ]
    }}

    ### JSON OUTPUT:
    """
)

chain_extract = prompt_extract | llm
result = chain_extract.invoke({"page_data": page_data})    
print(result.content)

```json
{
    "jobs": [
        {
            "role": "Generative AI Design Expert",
            "experience": "8+ years",
            "skills": ["Industrial Design", "Architecture", "Fashion Design", "Generative AI", "Visual Coding", "Additive Manufacturing", "3D Printing"],
            "description": "Nike is seeking a pioneering GenAI designer to create and implement AI-augmented workflows for footwear design. The ideal candidate will have a deep understanding of design principles and technical mastery of generative AI. They will partner with designers and engineers to transform Nike's approach to product creation."
        }
    ]
}
```


In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_result = json_parser.parse(result.content)
json_result


{'jobs': [{'role': 'Generative AI Design Expert',
   'experience': '8+ years',
   'skills': ['Industrial Design',
    'Architecture',
    'Fashion Design',
    'Generative AI',
    'Visual Coding',
    'Additive Manufacturing',
    '3D Printing'],
   'description': "Nike is seeking a pioneering GenAI designer to create and implement AI-augmented workflows for footwear design. The ideal candidate will have a deep understanding of design principles and technical mastery of generative AI. They will partner with designers and engineers to transform Nike's approach to product creation."}]}

In [6]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [7]:
import chromadb
import uuid

client = chromadb.PersistentClient("vector_store")
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [9]:
job = json_result['jobs'][0]
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [10]:
job = json_result['jobs'][0]
job['skills']

['Industrial Design',
 'Architecture',
 'Fashion Design',
 'Generative AI',
 'Visual Coding',
 'Additive Manufacturing',
 '3D Printing']

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_email = PromptTemplate.from_template(
    """
    ### ROLE:
    You are Mustafa, a Senior Business Development Executive at **Synthetix AI**. 
    Synthetix AI is a boutique AI Engineering firm specializing in LLM Orchestration, 
    Agentic Workflows, and Enterprise-grade Automation.

    ### INPUT DATA:
    JOB DESCRIPTION: 
    {job_description}

    PORTFOLIO LINKS: 
    {link_list}

    ### TASK:
    Write a high-converting, personalized cold email to the hiring manager/client. 
    Your goal is to demonstrate how Synthetix AI can solve the specific challenges mentioned in the job description.

    ### STRATEGY (The "Problem-Solution-Proof" Framework):
    1. **The Hook:** Start with a brief observation about their specific needs (from the job description).
    2. **The Value:** Position Synthetix AI as a partner that has solved similar problems using AI & Automation.
    3. **The Proof:** Select ONLY the top 2 most relevant links from the provided Portfolio Links that match the job's tech stack or goals.
    4. **The CTA:** A low-friction call to action (e.g., a 10-minute brainstorming call).

    ### CONSTRAINTS:
    - Persona: Professional, tech-savvy, helpful, and concise.
    - Avoid buzzwords like "revolutionary" or "game-changing".
    - Maximum length: 150 words.
    - No preamble (Don't say "Sure, here is the email").
    - Signature: Mustafa | Business Development, Synthetix AI.

    ### EMAIL OUTPUT:
    """
)

chain_email = prompt_email | llm
result = chain_email.invoke({"job_description": str(job), "link_list": links})
print(result.content)

KeyError: "Input to PromptTemplate is missing variables {'link_list'}.  Expected: ['job_description', 'link_list'] Received: ['job_description', 'link_lists']\nNote: if you intended {link_list} to be part of the string and not a variable, please escape it with double curly braces like: '{{link_list}}'.\nFor troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/INVALID_PROMPT_INPUT "